In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
root_dir = "/content/drive/MyDrive/"
base_dir = root_dir + "tesi_magistrale/datasets/to_zip/"
path_corpus_data = root_dir + "tesi_magistrale/datasets/corpus_data_folder/"
bbn_path = base_dir + "BBN/BBN/"

Mounted at /content/drive/


In [2]:
import json

with open(bbn_path + "train.json", 'r') as inp:
    examples_train = [json.loads(l) for l in inp.readlines()]

In [3]:
print("il dataset train contiene ",len(examples_train), "elementi")

il dataset train contiene  32739 elementi


In [ ]:
# print(len(examples_train[14535]['mentions']))
# print(len(examples_train[14535]['tokens']))
# prod = 1
# sum=0
# for elem in examples_train[14535]['mentions']:
#    prod *= len(elem)
#    sum+=len(elem)

# print(prod)
# print(sum/len(examples_train[14535]['mentions']))

27
77
7625597484987
3.0


In [ ]:
#del examples_train[14000:15000]

In [4]:
# rimuovo elemento 14535, caso problematico in cui generare tutte le permutazioni della frase avrebbe generato 7625597484987 frasi.
# questo elemento è una frase con 77 tokens, di cui 27 menzioni con ciascuna in media 3 tipi
del examples_train[14535]
examples_iter = iter(examples_train)

In [ ]:
#examples_iter = iter(examples_train[len(examples_train)//2:len(examples_train)+1])

In [5]:
def skip_this_phrase(mentions,dictionary):
    prod = 1
    for elem in mentions:
      prod*=len(elem['labels'])

    if prod<=10:
      dictionary['minori=_10']+=1
    if prod>10 and  prod<=20:
      dictionary['maggiori_10_minori=20']+=1
    if prod>20 and prod<=50:
      dictionary['maggiori_20_minori=50']+=1
    if prod>50 and prod<=100:
      dictionary['maggiori_50_minori=100']+=1
    if prod>100 and prod<=200:
      dictionary['maggiori_100_minori=200']+=1
    if (prod>200):
      dictionary['maggiori_200']+=1
    if (prod>20):
      return 0
    
    return prod

In [6]:
import operator
import itertools
from tqdm import tqdm
from gensim.parsing.preprocessing import remove_stopwords

perm_a_priori=[]
perm_effettive=[]
sottostime=[]
lunghezze={
    'minori=_10':0,
    'maggiori_10_minori=20':0,
    'maggiori_20_minori=50':0,
    'maggiori_50_minori=100':0,
    'maggiori_100_minori=200':0,
    'maggiori_200':0,
    'sottostimate': 0,
    'sovrastimate': 0,
    'frasi_con_overlap':0,
}

fp = open(path_corpus_data + 'corpus_data_entire_bbn.txt', "w")
restricted_set = set()

for num,i in enumerate(tqdm(examples_iter)):

  perm_priori=skip_this_phrase(i['mentions'],lunghezze)
  if (perm_priori==0):
    continue

  perm_a_priori.append(perm_priori)
  istance = i['mentions'][:]
  if len(istance)>1:

    d = sorted(istance, key=operator.itemgetter("start"))

    d = {k: list(v) for k, v in itertools.groupby(d, key=operator.itemgetter("start"))}
    all_list=[]
    for key, value in d.items():
      all_list.append(value)

    permutations = list(itertools.product(*all_list))

    iter = 0
    real_perm = 0
    for p in permutations:
          mini_list = []
          for lista_labels in p:
            mini_list.append(lista_labels['labels'])

          permutations_2 = itertools.product(*mini_list)
          real_perm+=len(list(permutations_2))
          for p_2 in permutations_2:
            
            phrase = i['tokens'][:]
            
            num_pop=0
            for h in range(0, len(p_2)):
              start = permutations[iter][h]['start']
              end = permutations[iter][h]['end']
      
              phrase[start-num_pop:end-num_pop] = [p_2[h]]
              if((end-start)>1):
                num_pop += (end - start) - 1

              restricted_set.add(p_2[h])

              
            fp.write((remove_stopwords(' '.join(phrase)) + '\n'))
          iter+=1
    if real_perm!=perm_priori:
      perm_a_priori.append(perm_priori)
      perm_effettive.append(real_perm)
      lunghezze['frasi_con_overlap']+=1
      if real_perm>perm_priori:
         sottostime.append(real_perm/perm_priori)
         lunghezze['sottostimate']+=1
      if real_perm<perm_priori:
         lunghezze['sovrastimate']+=1
      #print(f"Frase {num}, permutazioni a priori {perm_priori}, permutazioni effettive {real_perm}")
  else:
    for k in istance[0]['labels']:
      phrase = i['tokens'][:]
      if (i['mentions'][0]['end'] - i['mentions'][0]['start'] ==1):          
        phrase[i['mentions'][0]['start']] = k
        restricted_set.add(k)
      else:
        phrase[i['mentions'][0]['start']:i['mentions'][0]['end']] = [k]
        restricted_set.add(k)

      fp.write((remove_stopwords(' '.join(phrase)) + '\n'))

fp.close()

32738it [00:00, 79353.13it/s]


In [7]:
import statistics
print(f"frasi con overlap in BBN {lunghezze['frasi_con_overlap']}")
print(f"con stima a priori maggiore di effettiva {lunghezze['sovrastimate']}")
print(f"con stima a priori minore di effettiva {lunghezze['sottostimate']}")
print(f"mediana delle stime a priori {statistics.median(perm_a_priori)}")
print(f"mediana delle permutazioni effettive {statistics.median(perm_effettive)}")
print(f"nei {lunghezze['sottostimate']} casi in cui ho sottostimato la media della sottostima è di {sum(sottostime)/len(sottostime)}")

frasi con overlap in BBN 1131
con stima a priori maggiore di effettiva 307
con stima a priori minore di effettiva 824
mediana delle stime a priori 3
mediana delle permutazioni effettive 6
nei 824 casi in cui ho sottostimato la media della sottostima è di 1.6960558252427178


In [8]:
print(examples_train[28292])

{'tokens': ['Mr.', 'Buffett', "'s", 'firm', ',', 'Berkshire', 'Hathaway', ',', 'holds', 'a', 'much', 'bigger', 'stake', 'in', 'the', 'carrier', 'than', 'Mr.', 'Steinhardt', "'s", 'firm', ',', 'Steinhardt', 'Partners', '.'], 'senid': 5, 'mentions': [{'start': 5, 'labels': ['/ORGANIZATION/CORPORATION', '/ORGANIZATION'], 'end': 7}, {'start': 22, 'labels': ['/PERSON'], 'end': 24}, {'start': 1, 'labels': ['/PERSON'], 'end': 2}, {'start': 22, 'labels': ['/PERSON'], 'end': 23}], 'fileid': 'WSJ1881'}


In [ ]:
restricted_list = list(restricted_set)

In [ ]:
import pickle
with open(path_corpus_data + 'restricted_list_bbn.pkl', 'wb') as fp:
  pickle.dump(restricted_list, fp)